# Data Agregation and cleaning
Let's start by comparing the Load tab on excel file to the fact table on the warehouse. If these are equal then we move on to check the calculations tables

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sq
import pyodbc

In [ ]:
server = '####'
database = '####'
username = '####'
password = '####'
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
with cnxn:
    with cnxn.cursor() as crs:
        string = ('select #####')
    crs.execute(string)

In [ ]:
fact_df = pd.read_sql(string,con=cnxn)
fact_df[['Total','Avg']] = fact_df[['Total','Avg']].round(0).astype('int32')
fact_df


In [ ]:
load_df = pd.read_excel(io='Totals from Load Tab.xlsx',sheet_name='LoadTotals').drop('Unnamed: 0',axis=1).rename({'weekend_date':'Weeks'}, axis=1)
load_df

In [ ]:
compare_df = fact_df.merge(load_df[['weekend_year','weekend_month','All Campuses']],left_on=['ServiceYear','ServiceMonth'], right_on=['weekend_year','weekend_month'])
compare_df.drop(['weekend_year','weekend_month'],axis=1)
compare_df['diff'] = compare_df['Avg'] - compare_df['All Campuses']

In [ ]:
errors_metrics = compare_df[compare_df['Avg']!=compare_df['All Campuses']]

In [ ]:
errors_metrics

## Observations

We can see that the error metrics are very small and due to the rounding up during the load calculations. The error is of 0.001% and is acceptable.

Let's now move to observe load vs calculations tables

In [ ]:
string = ('select #####')
with cnxn:
    with cnxn.cursor():
        cursor.execute(string)

In [ ]:
summary_month_avg = pd.read_sql(string,con=cnxn)
summary_month_avg.columns = ['Year','Month','Avg','Weeks']

In [ ]:
summary_month_avg

In [ ]:
compare_summary_df = summary_month_avg.merge(load_df[['weekend_year','weekend_month','Weeks','All Campuses']],left_on=['Year','Month'], right_on=['weekend_year','weekend_month'])
compare_summary_df.drop(['weekend_year','weekend_month'],axis=1)
compare_summary_df['diff'] = compare_summary_df['Avg'] - compare_summary_df['All Campuses']

In [ ]:
errors_summary = compare_summary_df[~compare_summary_df['diff'].isin([1,-1])]
errors_summary

In [ ]:
writer = pd.ExcelWriter('errors in warehouse.xlsx',engine='xlsxwriter')
errors_metrics.to_excel(writer,sheet_name='Errors in Metrics')
errors_summary.to_excel(writer,sheet_name='Errors Summary Monthly Avg')
writer.save()

## Observations

Although the numbers on FactMetrics are correct, the calculations table for monthly averages seem off. This is already taking into account the volunteer numbers of the international campuses
